# Fore-02 Generate full query

El objetivo de este notebook es testear la query definida para descargar el dataset full definido para los experimentos de forecasting de multiwarehouse. Específicamente nos interesa obtener todas las compras de un sitio dado (por ejemplo, MLB) y en un rango de fechas, y de ellas además de la información en la query básica, también queremos los siguientes campos:

* Precio del item
* Título del item al momento de la compra
* Dominio
* Marca
* Modelo
* Categoría
* Product_id
* Logística del envío
* Origen
* Seller_id
* Buyer_id
* Precio del envío
* Promesa de entrega (speed y offset)

Nuevamente, para los datos descargados, si la compra se realizó con Mercado Envíos existe un shipment_id y otra información del envío. De lo contrario, dichos campos quedan nulos.

## Importaciones y definiciones

In [ ]:
import os

from datetime import datetime
from pathlib import Path

from ml_forecasting.etl.base import FullExperimentDataSource
from ml_forecasting.etl.dataset import DataPreparation
from ml_forecasting.config import CONFIGS_BY_SITE

In [ ]:
# parameters
data_root = Path.home().joinpath('/alloc/data/fury_ml-forecasting/etl/raw_data/')
site = 'MLB'
teradata_user = os.environ['SECRET_TERADATA_USER']
teradata_pass = os.environ['SECRET_TERADATA_PASS']

## Desarrollo

En primer lugar definimos el período de tiempo de compras que queremos descargar y chequeamos si el path donde vamos a guardar la información existe:

In [ ]:
start_download = datetime.strptime('2019-09-07', '%Y-%m-%d')
end_download = datetime.strptime('2019-09-14', '%Y-%m-%d')

In [ ]:
data_root.mkdir(exist_ok=True)

Descargamos los datos empleando la query definida en `ml_forecasting.etl.base`:

In [ ]:
source = FullExperimentDataSource(teradata_user, teradata_pass, data_root, site)
files = source.csvfiles(start_download, end_download, delta=(end_download - start_download).days)

In [ ]:
for f in files:
    print('*', f)

Tenemos todos los datos guardados en el archivo, ahora vamos a llamar a `DataPreparation` del módulo `ml_forecasting.etl.dataset` para cargar y preparar el dataframe:

In [ ]:
prep = DataPreparation(CONFIGS_BY_SITE[site])

In [ ]:
df = prep.load(files, cols=CONFIGS_BY_SITE[site]['DTYPES'])

Veamos el tamaño y un ejemplo del dataframe resultante:

In [ ]:
df.shape

In [ ]:
df.sample(5)

Comparando con la query básica tenemos más filas porque puede haber dos atributos (marca y modelo) para el mismo item y orden. Luego, debemos filtrar por `ord_order_id` diferentes:

In [ ]:
diff_buys = df.drop_duplicates(subset='ord_order_id')
print("Cantidad de órdenes diferentes: {}".format(diff_buys.shape[0]))

Cuando [descargamos los datos utilizando la query básica](../reports/191009_Fore-01_Generate_basic_query.html) teníamos 3392551 filas, donde cada fila corresponde a una orden. Luego, vemos que en la query full obtenemos la misma cantidad de datos.

Por último, podemos comparar la cantidad de SI (items vendidos) que obtenemos a partir de la query generada con los datos registrados en el [dashboard de PortalBI](http://portalbi.ml.com/#/business_unit/2/dashboards/67) (en este caso, 4021412 SI):

In [ ]:
SI_query = diff_buys.bid_quantity_ok.sum()
print("Cantidad de SI descargados en la query: {}".format(SI_query))

In [ ]:
SI_BI = 4021412
print("Porcentaje de diferencia entre la query y el dashboard de BI: {:.3f}%".format((SI_query-SI_BI)/SI_BI*100))

Podemos ver que la diferencia es menor al 0.01%, por lo cual asumimos que la query generada es correcta y se obtienen todas las compras realizadas en el período de interés.

## Conclusión

El objetivo de este notebook era testear la query definida para descargar el dataset full definido para los experimentos de forecasting de multiwarehouse. Utilizando la query definida en `ml_forecasting.etl.base` pudimos descargar los datos de interés de las compras de MLB para un rango de fechas (además de aquellos que ya se descargan en la query básica). Por otra parte, con la clase `DataPreparation` definida en `ml_forecasting.etl.dataset` pudimos cargar y preparar el dataframe. De esta manera, seteamos las bases (query y preparación de datos) para poder obtener los datos a utilizar en los futuros análisis de forecasting.